In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("pdf_image_loader")

LangSmith 추적을 시작합니다.
[프로젝트명]
pdf_image_loader


In [3]:
!pip install -qU langchain-teddynote

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opentelemetry-proto 1.28.2 requires protobuf<6.0,>=5.0, but you have protobuf 4.25.5 which is incompatible.


## 저장할 State 정의

In [4]:
from typing import TypedDict


# GraphState 상태를 저장하는 용도로 사용합니다.
class GraphState(TypedDict):
    filepath: str  # path
    filetype: str  # pdf
    page_numbers: list[int]  # page numbers
    batch_size: int  # batch size
    split_filepaths: list[str]  # split files
    analyzed_files: list[str]  # analyzed files
    page_elements: dict[int, dict[str, list[dict]]]  # page elements
    page_metadata: dict[int, dict]  # page metadata
    page_summary: dict[int, str]  # page summary
    images: list[str]  # image paths
    images_summary: list[str]  # image summary
    tables: list[str]  # table
    tables_summary: dict[int, str]  # table summary
    texts: list[str]  # text
    texts_summary: list[str]  # text summary

## 문서를 배치 단위로 분할(10 page)

In [5]:
import os
import pymupdf
import json
import requests
from PIL import Image
import fitz


In [10]:
def split_pdf(state: GraphState):
    """
    입력 PDF를 여러 개의 작은 PDF 파일로 분할합니다.

    :param state: GraphState 객체, PDF 파일 경로와 배치 크기 정보를 포함
    :return: 분할된 PDF 파일 경로 목록을 포함한 GraphState 객체
    """
    # PDF 파일 경로와 배치 크기 추출
    filepath = state["filepath"]
    batch_size = state["batch_size"]

    # PDF 파일 열기
    input_pdf = fitz.open(filepath)
    num_pages = len(input_pdf)
    print(f"총 페이지 수: {num_pages}")

    ret = []
    # PDF 분할 작업 시작
    for start_page in range(0, num_pages, batch_size):
        # 배치의 마지막 페이지 계산 (전체 페이지 수를 초과하지 않도록)
        end_page = min(start_page + batch_size, num_pages) - 1

        # 분할된 PDF 파일명 생성
        input_file_basename = os.path.splitext(filepath)[0]
        output_file = f"{input_file_basename}_{start_page:04d}_{end_page:04d}.pdf"
        print(f"분할 PDF 생성: {output_file}")

        # 새로운 PDF 파일 생성 및 페이지 삽입
        with fitz.open() as output_pdf:
            output_pdf.insert_pdf(input_pdf, from_page=start_page, to_page=end_page)
            output_pdf.save(output_file)
            ret.append(output_file)

    # 원본 PDF 파일 닫기
    input_pdf.close()

    # 분할된 PDF 파일 경로 목록을 포함한 GraphState 객체 반환
    return GraphState(split_filepaths=ret)

## state 등록

In [7]:
# # 모든 PDF 파일 경로를 가져오는 함수
# def get_pdf_files_in_directory(directory_path):
#     # 디렉토리 내 모든 파일 중 PDF 파일만 필터링하여 리스트로 반환
#     return [f for f in os.listdir(directory_path) if f.endswith(".pdf")]

# # 입력 PDF 디렉토리 경로
# input_directory_path = r"C:\Users\user\working\GitHub\RePick-MLOps\data\pdf"

# # 출력 경로
# output_directory_path = "data/"

# # PDF 파일 리스트 가져오기
# pdf_files = get_pdf_files_in_directory(input_directory_path)

# # 상태 객체 초기화
# state = GraphState()

# # 각 PDF 파일을 처리하는 코드
# for pdf_file in pdf_files:
#     file_path = os.path.join(input_directory_path, pdf_file)  # 전체 입력 파일 경로
#     # 기존 상태의 filepath와 batch_size만 갱신
#     state["filepath"] = file_path
#     state["batch_size"] = 1

#     # PDF 분할 수행
#     state_out = split_pdf(state)

#     # split_filepaths가 이미 state에 존재하면 추가, 없으면 초기화
#     state.setdefault("split_filepaths", [])
#     state["split_filepaths"].extend(state_out["split_filepaths"])

# # 마지막에 모든 처리된 state 출력
# state

{}

In [11]:
state = GraphState(filepath="data/pdf/20241122_company_429942000.pdf", batch_size=1)
state_out = split_pdf(state)
state.update(state_out)
state

총 페이지 수: 4
분할 PDF 생성: data/pdf/20241122_company_429942000_0000_0000.pdf
분할 PDF 생성: data/pdf/20241122_company_429942000_0001_0001.pdf
분할 PDF 생성: data/pdf/20241122_company_429942000_0002_0002.pdf
분할 PDF 생성: data/pdf/20241122_company_429942000_0003_0003.pdf


{'filepath': 'data/pdf/20241122_company_429942000.pdf',
 'batch_size': 1,
 'split_filepaths': ['data/pdf/20241122_company_429942000_0000_0000.pdf',
  'data/pdf/20241122_company_429942000_0001_0001.pdf',
  'data/pdf/20241122_company_429942000_0002_0002.pdf',
  'data/pdf/20241122_company_429942000_0003_0003.pdf']}

## Layout Analyzer를 사용하여 문서를 element 단위로 분할

In [12]:
class LayoutAnalyzer:
    def __init__(self, api_key):
        """
        LayoutAnalyzer 클래스의 생성자

        :param api_key: Upstage API 인증을 위한 API 키
        """
        self.api_key = api_key

    def _upstage_layout_analysis(self, input_file):
        """
        Upstage의 레이아웃 분석 API를 호출하여 문서 분석을 수행합니다.

        :param input_file: 분석할 PDF 파일의 경로
        :return: 분석 결과가 저장된 JSON 파일의 경로
        """
        # API 요청 헤더 설정
        headers = {"Authorization": f"Bearer {self.api_key}"}

        # API 요청 데이터 설정 (OCR 비활성화)
        data = {"ocr": True}

        # 분석할 PDF 파일 열기
        files = {"document": open(input_file, "rb")}

        # API 요청 보내기
        response = requests.post(
            "https://api.upstage.ai/v1/document-ai/layout-analysis",
            headers=headers,
            data=data,
            files=files,
        )

        # API 응답 처리 및 결과 저장
        if response.status_code == 200:
            # 분석 결과를 저장할 JSON 파일 경로 생성
            output_file = os.path.splitext(input_file)[0] + ".json"

            # 분석 결과를 JSON 파일로 저장
            with open(output_file, "w") as f:
                json.dump(response.json(), f, ensure_ascii=False)

            return output_file
        else:
            # API 요청이 실패한 경우 예외 발생
            raise ValueError(f"API 요청 실패. 상태 코드: {response.status_code}")

    def execute(self, input_file):
        """
        주어진 입력 파일에 대해 레이아웃 분석을 실행합니다.

        :param input_file: 분석할 PDF 파일의 경로
        :return: 분석 결과가 저장된 JSON 파일의 경로
        """
        return self._upstage_layout_analysis(input_file)

In [13]:
def analyze_layout(state: GraphState):
    # 분할된 PDF 파일 목록을 가져옵니다.
    split_files = state["split_filepaths"]

    # LayoutAnalyzer 객체를 생성합니다. API 키는 환경 변수에서 가져옵니다.
    analyzer = LayoutAnalyzer(os.environ.get("UPSTAGE_API_KEY"))

    # 분석된 파일들의 경로를 저장할 리스트를 초기화합니다.
    analyzed_files = []

    # 각 분할된 PDF 파일에 대해 레이아웃 분석을 수행합니다.
    for file in split_files:
        # 레이아웃 분석을 실행하고 결과 파일 경로를 리스트에 추가합니다.
        analyzed_files.append(analyzer.execute(file))

    # 분석된 파일 경로들을 정렬하여 새로운 GraphState 객체를 생성하고 반환합니다.
    # 정렬은 파일들의 순서를 유지하기 위해 수행됩니다.
    return GraphState(analyzed_files=sorted(analyzed_files))

In [14]:
state_out = analyze_layout(state)
state.update(state_out)
state

{'filepath': 'data/pdf/20241122_company_429942000.pdf',
 'batch_size': 1,
 'split_filepaths': ['data/pdf/20241122_company_429942000_0000_0000.pdf',
  'data/pdf/20241122_company_429942000_0001_0001.pdf',
  'data/pdf/20241122_company_429942000_0002_0002.pdf',
  'data/pdf/20241122_company_429942000_0003_0003.pdf'],
 'analyzed_files': ['data/pdf/20241122_company_429942000_0000_0000.json',
  'data/pdf/20241122_company_429942000_0001_0001.json',
  'data/pdf/20241122_company_429942000_0002_0002.json',
  'data/pdf/20241122_company_429942000_0003_0003.json']}

## 페이지 메타데이터 추출

In [16]:
import re
import glob


def extract_start_end_page(filename):
    """
    파일 이름에서 시작 페이지와 끝 페이지 번호를 추출하는 함수입니다.

    :param filename: 분석할 파일의 이름
    :return: 시작 페이지 번호와 끝 페이지 번호를 튜플로 반환
    """
    # 파일 경로에서 파일 이름만 추출
    file_name = os.path.basename(filename)
    # 파일 이름을 '_' 기준으로 분리
    file_name_parts = file_name.split("_")

    if len(file_name_parts) >= 3:
        # 파일 이름의 뒤에서 두 번째 부분에서 숫자를 추출하여 시작 페이지로 설정
        start_page = int(re.findall(r"(\d+)", file_name_parts[-2])[0])
        # 파일 이름의 마지막 부분에서 숫자를 추출하여 끝 페이지로 설정
        end_page = int(re.findall(r"(\d+)", file_name_parts[-1])[0])
    else:
        # 파일 이름 형식이 예상과 다를 경우 기본값 설정
        start_page, end_page = 0, 0

    return start_page, end_page


def extract_page_metadata(state: GraphState):
    """
    분석된 JSON 파일들에서 페이지 메타데이터를 추출하는 함수입니다.

    :param state: 현재의 GraphState 객체
    :return: 페이지 메타데이터가 추가된 새로운 GraphState 객체
    """
    
    # 분석된 JSON 파일 목록 가져오기
    json_files = state["analyzed_files"]

    # 페이지 메타데이터를 저장할 딕셔너리 초기화
    page_metadata = dict()

    for json_file in json_files:
        # JSON 파일 열기 및 데이터 로드
        with open(json_file, "r") as f:
            data = json.load(f)

        # 파일명에서 시작 페이지 번호 추출
        start_page, _ = extract_start_end_page(json_file)

        # JSON 데이터에서 각 페이지의 메타데이터 추출
        for element in data["metadata"]["pages"]:
            # 원본 페이지 번호
            original_page = int(element["page"])
            # 상대적 페이지 번호 계산 (전체 문서 기준)
            relative_page = start_page + original_page - 1

            # 페이지 크기 정보 추출
            metadata = {
                "size": [
                    int(element["width"]),
                    int(element["height"]),
                ],
            }
            # 상대적 페이지 번호를 키로 하여 메타데이터 저장
            page_metadata[relative_page] = metadata

    # 추출된 페이지 메타데이터로 새로운 GraphState 객체 생성 및 반환
    return GraphState(page_metadata=page_metadata)

페이지별 사이즈를 추출합니다.

In [17]:
state_out = extract_page_metadata(state)
state.update(state_out)
state["page_metadata"]

{0: {'size': [1241, 1754]},
 1: {'size': [1241, 1754]},
 2: {'size': [1241, 1754]},
 3: {'size': [1241, 1754]}}

## 페이지별 HTML Element 추출

In [18]:
def extract_page_elements(state: GraphState):
    # 분석된 JSON 파일 목록을 가져옵니다.
    json_files = state["analyzed_files"]

    # 페이지별 요소를 저장할 딕셔너리를 초기화합니다.
    page_elements = dict()

    # 전체 문서에서 고유한 요소 ID를 부여하기 위한 카운터입니다.
    element_id = 0

    # 각 JSON 파일을 순회하며 처리합니다.
    for json_file in json_files:
        # 파일명에서 시작 페이지 번호를 추출합니다.
        start_page, _ = extract_start_end_page(json_file)

        # JSON 파일을 열어 데이터를 로드합니다.
        with open(json_file, "r") as f:
            data = json.load(f)

        # JSON 데이터의 각 요소를 처리합니다.
        for element in data["elements"]:
            # 원본 페이지 번호를 정수로 변환합니다.
            original_page = int(element["page"])
            # 전체 문서 기준의 상대적 페이지 번호를 계산합니다.
            relative_page = start_page + original_page - 1

            # 해당 페이지의 요소 리스트가 없으면 새로 생성합니다.
            if relative_page not in page_elements:
                page_elements[relative_page] = []

            # 요소에 고유 ID를 부여합니다.
            element["id"] = element_id
            element_id += 1

            # 요소의 페이지 번호를 상대적 페이지 번호로 업데이트합니다.
            element["page"] = relative_page
            # 요소를 해당 페이지의 리스트에 추가합니다.
            page_elements[relative_page].append(element)

    # 추출된 페이지별 요소 정보로 새로운 GraphState 객체를 생성하여 반환합니다.
    return GraphState(page_elements=page_elements)

In [19]:
state_out = extract_page_elements(state)
state.update(state_out)
state["page_elements"][1]

[{'bounding_box': [{'x': 83, 'y': 69},
   {'x': 151, 'y': 69},
   {'x': 151, 'y': 95},
   {'x': 83, 'y': 95}],
  'category': 'paragraph',
  'html': "<p id='0' data-category='paragraph' style='font-size:22px'>코리안리</p>",
  'id': 16,
  'page': 1,
  'text': '코리안리'},
 {'bounding_box': [{'x': 1059, 'y': 69},
   {'x': 1155, 'y': 69},
   {'x': 1155, 'y': 95},
   {'x': 1059, 'y': 95}],
  'category': 'header',
  'html': "<br><header id='1' style='font-size:16px'>2024.11.22</header>",
  'id': 17,
  'page': 1,
  'text': '2024.11.22'},
 {'bounding_box': [{'x': 379, 'y': 181},
   {'x': 541, 'y': 181},
   {'x': 541, 'y': 206},
   {'x': 379, 'y': 206}],
  'category': 'caption',
  'html': "<caption id='2' style='font-size:20px'>표 1. 목표주가 산출</caption>",
  'id': 18,
  'page': 1,
  'text': '표 1. 목표주가 산출'},
 {'bounding_box': [{'x': 1077, 'y': 181},
   {'x': 1155, 'y': 181},
   {'x': 1155, 'y': 207},
   {'x': 1077, 'y': 207}],
  'category': 'paragraph',
  'html': "<br><p id='3' data-category='paragraph' sty

추출된 페이지를 확인

In [20]:
state["page_elements"].keys()

dict_keys([0, 1, 2, 3])

## 각 페이지의 태그를 분해합니다.
- images, tables, text를 분해합니다.

In [21]:
def extract_tag_elements_per_page(state: GraphState):
    # GraphState 객체에서 페이지 요소들을 가져옵니다.
    page_elements = state["page_elements"]

    # 파싱된 페이지 요소들을 저장할 새로운 딕셔너리를 생성합니다.
    parsed_page_elements = dict()

    # 각 페이지와 해당 페이지의 요소들을 순회합니다.
    for key, page_element in page_elements.items():
        # 이미지, 테이블, 텍스트 요소들을 저장할 리스트를 초기화합니다.
        image_elements = []
        table_elements = []
        text_elements = []

        # 페이지의 각 요소를 순회하며 카테고리별로 분류합니다.
        for element in page_element:
            if element["category"] == "chart":
                # 이미지 요소인 경우 image_elements 리스트에 추가합니다.
                image_elements.append(element)
            elif element["category"] == "table":
                # 테이블 요소인 경우 table_elements 리스트에 추가합니다.
                table_elements.append(element)
            else:
                # 그 외의 요소는 모두 텍스트 요소로 간주하여 text_elements 리스트에 추가합니다.
                text_elements.append(element)

        # 분류된 요소들을 페이지 키와 함께 새로운 딕셔너리에 저장합니다.
        parsed_page_elements[key] = {
            "image_elements": image_elements,
            "table_elements": table_elements,
            "text_elements": text_elements,
            "elements": page_element,  # 원본 페이지 요소도 함께 저장합니다.
        }

    # 파싱된 페이지 요소들을 포함한 새로운 GraphState 객체를 반환합니다.
    return GraphState(page_elements=parsed_page_elements)

In [22]:
state_out = extract_tag_elements_per_page(state)
state.update(state_out)

In [23]:
state["page_elements"].keys()

dict_keys([0, 1, 2, 3])

In [24]:
state["page_elements"][1].keys()

dict_keys(['image_elements', 'table_elements', 'text_elements', 'elements'])

In [29]:
state["page_elements"][2]["image_elements"]

[]

In [28]:
state["page_elements"][3]["table_elements"]

[{'bounding_box': [{'x': 78, 'y': 210},
   {'x': 1156, 'y': 210},
   {'x': 1156, 'y': 462},
   {'x': 78, 'y': 462}],
  'category': 'table',
  'html': '<br><table id=\'3\' style=\'font-size:14px\'><tr><td rowspan="2">제시일자</td><td rowspan="2">투자의견</td><td rowspan="2">목표주가(원)</td><td colspan="2">괴리율(%)</td><td rowspan="2">(원) 코리 안리 12,000</td></tr><tr><td>평균주가대비</td><td>최고(최저)주가대비</td></tr><tr><td colspan="5">코리안리 (003690)</td><td>10,000</td></tr><tr><td>2024. 11 .22</td><td>매수</td><td>10,000</td><td>-</td><td></td><td>8,000</td></tr><tr><td>2024.06. 10</td><td>매수</td><td>8,487</td><td>-15.06</td><td>-4.21</td><td></td></tr><tr><td>2008.01.31</td><td>1년 경과 이후</td><td></td><td>-</td><td></td><td>6,000 4,000 2,000 0 22.11 23.11 24.11</td></tr></table>',
  'id': 39,
  'page': 3,
  'text': '제시일자 투자의견 목표주가(원) 괴리율(%) (원) 코리 안리 12,000\n 평균주가대비 최고(최저)주가대비\n 코리안리 (003690) 10,000\n 2024. 11 .22 매수 10,000 -  8,000\n 2024.06. 10 매수 8,487 -15.06 -4.21 \n 2008.01.31 1년 경과 이후  -  6,000 4,000 2,000 0 22.

In [499]:
state["page_elements"][2]["text_elements"]

[{'bounding_box': [{'x': 80, 'y': 127},
   {'x': 247, 'y': 127},
   {'x': 247, 'y': 159},
   {'x': 80, 'y': 159}],
  'category': 'heading1',
  'html': "<h1 id='0' style='font-size:20px'>EMB(278990)</h1>",
  'id': 23,
  'page': 2,
  'text': 'EMB(278990)'},
 {'bounding_box': [{'x': 1165, 'y': 1670},
   {'x': 1184, 'y': 1670},
   {'x': 1184, 'y': 1693},
   {'x': 1165, 'y': 1693}],
  'category': 'footer',
  'html': "<footer id='2' style='font-size:14px'>2</footer>",
  'id': 25,
  'page': 2,
  'text': '2'},
 {'bounding_box': [{'x': 79, 'y': 126},
   {'x': 283, 'y': 126},
   {'x': 283, 'y': 160},
   {'x': 79, 'y': 160}],
  'category': 'heading1',
  'html': "<h1 id='0' style='font-size:20px'>엘리비젼(276240)</h1>",
  'id': 218,
  'page': 2,
  'text': '엘리비젼(276240)'},
 {'bounding_box': [{'x': 1166, 'y': 1671},
   {'x': 1184, 'y': 1671},
   {'x': 1184, 'y': 1693},
   {'x': 1166, 'y': 1693}],
  'category': 'footer',
  'html': "<br><footer id='2' style='font-size:14px'>2</footer>",
  'id': 220,
  'pa

## 페이지 번호를 추출합니다.

In [30]:
def page_numbers(state: GraphState):
    return GraphState(page_numbers=list(state["page_elements"].keys()))


state_out = page_numbers(state)
state.update(state_out)
state["page_numbers"]

[0, 1, 2, 3]

## 이미지 추출합니다.

In [63]:
class ImageCropper:
    @staticmethod
    def pdf_to_image(pdf_file, page_num, dpi=300):
        """
        PDF 파일의 특정 페이지를 이미지로 변환하는 메서드

        :param page_num: 변환할 페이지 번호 (1부터 시작)
        :param dpi: 이미지 해상도 (기본값: 300)
        :return: 변환된 이미지 객체
        """
        with fitz.open(pdf_file) as doc:
            page = doc[page_num].get_pixmap(dpi=dpi)
            target_page_size = [page.width, page.height]
            page_img = Image.frombytes("RGB", target_page_size, page.samples)
        return page_img

    @staticmethod
    def normalize_coordinates(coordinates, output_page_size):
        """
        좌표를 정규화하는 정적 메서드

        :param coordinates: 원본 좌표 리스트
        :param output_page_size: 출력 페이지 크기 [너비, 높이]
        :return: 정규화된 좌표 (x1, y1, x2, y2)
        """
        x_values = [coord["x"] for coord in coordinates]
        y_values = [coord["y"] for coord in coordinates]
        x1, y1, x2, y2 = min(x_values), min(y_values), max(x_values), max(y_values)

        return (
            x1 / output_page_size[0],
            y1 / output_page_size[1],
            x2 / output_page_size[0],
            y2 / output_page_size[1],
        )

    @staticmethod
    def crop_image(img, coordinates, output_file):
        """
        이미지를 주어진 좌표에 따라 자르고 저장하는 정적 메서드

        :param img: 원본 이미지 객체
        :param coordinates: 정규화된 좌표 (x1, y1, x2, y2)
        :param output_file: 저장할 파일 경로
        """
        img_width, img_height = img.size
        x1, y1, x2, y2 = [
            int(coord * dim)
            for coord, dim in zip(coordinates, [img_width, img_height] * 2)
        ]
        cropped_img = img.crop((x1, y1, x2, y2))
        cropped_img.save(output_file)

In [64]:
sample_table = state["page_elements"][1]["table_elements"][0]
sample_table

{'bounding_box': [{'x': 381, 'y': 213},
  {'x': 1150, 'y': 213},
  {'x': 1150, 'y': 593},
  {'x': 381, 'y': 593}],
 'category': 'table',
 'html': "<br><table id='4' style='font-size:16px'><tr><td>항목</td><td>값</td><td>비고</td></tr><tr><td>기존 목표주가</td><td>10,000</td><td></td></tr><tr><td>2024F ROE</td><td>9.3</td><td>당사 추정치</td></tr><tr><td>수정 할인율</td><td>21.2</td><td>내재 할인율을 기준으로 당사 조정</td></tr><tr><td>목표 P/B</td><td>0.44</td><td></td></tr><tr><td>2024F BPS</td><td>22,556</td><td>당사 추정치, 2024년 무상증자 반영 전</td></tr><tr><td>신규 목표주가</td><td>10,000</td><td></td></tr><tr><td>2025F ROE</td><td>8.1</td><td>당사 추정치</td></tr><tr><td>수정 할인율</td><td>16.1</td><td>내재 할인율을 기준으로 당사 조정</td></tr><tr><td>목표 P/B</td><td>0.50</td><td></td></tr><tr><td>2025F BPS</td><td>19,871</td><td>당사 추정치, 2025년 예정 무상증자 반영 전</td></tr><tr><td>현재 주가</td><td>7,940</td><td>전일 종가 기준</td></tr><tr><td>상승여력</td><td>25.9</td><td></td></tr><tr><td>투자의견</td><td>매수</td><td>상승여력 20% 이상</td></tr></table>",
 'id': 20,
 'page': 1,
 'text': 

In [65]:
sample_table = state["page_elements"][1]["table_elements"][0]
sample_table

{'bounding_box': [{'x': 381, 'y': 213},
  {'x': 1150, 'y': 213},
  {'x': 1150, 'y': 593},
  {'x': 381, 'y': 593}],
 'category': 'table',
 'html': "<br><table id='4' style='font-size:16px'><tr><td>항목</td><td>값</td><td>비고</td></tr><tr><td>기존 목표주가</td><td>10,000</td><td></td></tr><tr><td>2024F ROE</td><td>9.3</td><td>당사 추정치</td></tr><tr><td>수정 할인율</td><td>21.2</td><td>내재 할인율을 기준으로 당사 조정</td></tr><tr><td>목표 P/B</td><td>0.44</td><td></td></tr><tr><td>2024F BPS</td><td>22,556</td><td>당사 추정치, 2024년 무상증자 반영 전</td></tr><tr><td>신규 목표주가</td><td>10,000</td><td></td></tr><tr><td>2025F ROE</td><td>8.1</td><td>당사 추정치</td></tr><tr><td>수정 할인율</td><td>16.1</td><td>내재 할인율을 기준으로 당사 조정</td></tr><tr><td>목표 P/B</td><td>0.50</td><td></td></tr><tr><td>2025F BPS</td><td>19,871</td><td>당사 추정치, 2025년 예정 무상증자 반영 전</td></tr><tr><td>현재 주가</td><td>7,940</td><td>전일 종가 기준</td></tr><tr><td>상승여력</td><td>25.9</td><td></td></tr><tr><td>투자의견</td><td>매수</td><td>상승여력 20% 이상</td></tr></table>",
 'id': 20,
 'page': 1,
 'text': 

## 이미지 추출

In [66]:
state["split_filepaths"]

['data/pdf/20241122_company_429942000_0000_0000.pdf',
 'data/pdf/20241122_company_429942000_0001_0001.pdf',
 'data/pdf/20241122_company_429942000_0002_0002.pdf',
 'data/pdf/20241122_company_429942000_0003_0003.pdf']

In [70]:
def crop_image(state: GraphState):
    """
    PDF 파일에서 이미지를 추출하고 크롭하는 함수

    :param state: GraphState 객체
    :return: 크롭된 이미지 정보가 포함된 GraphState 객체
    """
    pdf_file = state["filepath"]  # PDF 파일 경로
    page_numbers = state["page_numbers"]  # 처리할 페이지 번호 목록
    output_folder = os.path.splitext(pdf_file)[0]  # 출력 폴더 경로 설정
    os.makedirs(output_folder, exist_ok=True)  # 출력 폴더 생성

    cropped_images = dict()  # 크롭된 이미지 정보를 저장할 딕셔너리
    for page_num in page_numbers:
        pdf_image = ImageCropper.pdf_to_image(
            pdf_file, page_num
        )  # PDF 페이지를 이미지로 변환
        for element in state["page_elements"][page_num]["image_elements"]:
            if element["category"] == "figure":
                # 이미지 요소의 좌표를 정규화
                normalized_coordinates = ImageCropper.normalize_coordinates(
                    element["bounding_box"], state["page_metadata"][page_num]["size"]
                )

                # 크롭된 이미지 저장 경로 설정
                output_file = os.path.join(output_folder, f"{element['id']}.png")
                # 이미지 크롭 및 저장
                ImageCropper.crop_image(pdf_image, normalized_coordinates, output_file)
                cropped_images[element["id"]] = output_file
                print(f"page:{page_num}, id:{element['id']}, path: {output_file}")
    return GraphState(
        images=cropped_images
    )  # 크롭된 이미지 정보를 포함한 GraphState 반환


def crop_table(state: GraphState):
    """
    PDF 파일에서 표를 추출하고 크롭하는 함수

    :param state: GraphState 객체
    :return: 크롭된 표 이미지 정보가 포함된 GraphState 객체
    """
    pdf_file = state["filepath"]  # PDF 파일 경로
    page_numbers = state["page_numbers"]  # 처리할 페이지 번호 목록
    output_folder = os.path.splitext(pdf_file)[0]  # 출력 폴더 경로 설정
    os.makedirs(output_folder, exist_ok=True)  # 출력 폴더 생성

    cropped_images = dict()  # 크롭된 표 이미지 정보를 저장할 딕셔너리
    for page_num in page_numbers:
        pdf_image = ImageCropper.pdf_to_image(
            pdf_file, page_num
        )  # PDF 페이지를 이미지로 변환
        for element in state["page_elements"][page_num]["table_elements"]:
            if element["category"] == "table":
                # 표 요소의 좌표를 정규화
                normalized_coordinates = ImageCropper.normalize_coordinates(
                    element["bounding_box"], state["page_metadata"][page_num]["size"]
                )

                # 크롭된 표 이미지 저장 경로 설정
                output_file = os.path.join(output_folder, f"{element['id']}.png")
                # 표 이미지 크롭 및 저장
                ImageCropper.crop_image(pdf_image, normalized_coordinates, output_file)
                cropped_images[element["id"]] = output_file
                print(f"page:{page_num}, id:{element['id']}, path: {output_file}")
    return GraphState(
        tables=cropped_images
    )  # 크롭된 표 이미지 정보를 포함한 GraphState 반환

In [68]:
state["page_elements"].keys()

dict_keys([0, 1, 2, 3])

In [71]:
state_out = crop_image(state)
state.update(state_out)
state["images"]
# state.update(state6)

{}

In [72]:
state_out = crop_table(state)
state.update(state_out)
state["tables"]

page:0, id:2, path: data/pdf/20241122_company_429942000\2.png
page:0, id:14, path: data/pdf/20241122_company_429942000\14.png
page:1, id:20, path: data/pdf/20241122_company_429942000\20.png
page:1, id:24, path: data/pdf/20241122_company_429942000\24.png
page:1, id:28, path: data/pdf/20241122_company_429942000\28.png
page:2, id:34, path: data/pdf/20241122_company_429942000\34.png
page:3, id:39, path: data/pdf/20241122_company_429942000\39.png
page:3, id:42, path: data/pdf/20241122_company_429942000\42.png
page:3, id:44, path: data/pdf/20241122_company_429942000\44.png


{2: 'data/pdf/20241122_company_429942000\\2.png',
 14: 'data/pdf/20241122_company_429942000\\14.png',
 20: 'data/pdf/20241122_company_429942000\\20.png',
 24: 'data/pdf/20241122_company_429942000\\24.png',
 28: 'data/pdf/20241122_company_429942000\\28.png',
 34: 'data/pdf/20241122_company_429942000\\34.png',
 39: 'data/pdf/20241122_company_429942000\\39.png',
 42: 'data/pdf/20241122_company_429942000\\42.png',
 44: 'data/pdf/20241122_company_429942000\\44.png'}

## 페이지 텍스트 추출

In [37]:
def extract_page_text(state: GraphState):
    # 상태 객체에서 페이지 번호 목록을 가져옵니다.
    page_numbers = state["page_numbers"]

    # 추출된 텍스트를 저장할 딕셔너리를 초기화합니다.
    extracted_texts = dict()

    # 각 페이지 번호에 대해 반복합니다.
    for page_num in page_numbers:
        # 현재 페이지의 텍스트를 저장할 빈 문자열을 초기화합니다.
        extracted_texts[page_num] = ""

        # 현재 페이지의 모든 텍스트 요소에 대해 반복합니다.
        for element in state["page_elements"][page_num]["text_elements"]:
            # 각 텍스트 요소의 내용을 현재 페이지의 텍스트에 추가합니다.
            extracted_texts[page_num] += element["text"]

    # 추출된 텍스트를 포함한 새로운 GraphState 객체를 반환합니다.
    return GraphState(texts=extracted_texts)

In [38]:
state_out = extract_page_text(state)
state.update(state_out)
state["texts"]

{0: 'MIRAE ASSET\n미래에셋증권Equity Research\n2024. 11.22[금융]정태준, CFA\ntaejoon.jeong@miraeasset.com003690 · 보험코리안리매력적인 대안투자의견 매수, 목표주가 10,000원 제시코리안리에 대한 투자의견 매수, 목표주가 10,000원 제시. 이는 지난 11월 7일 무\n상증자 신주배정 기준일 이전 기준으로는 11,783원에 해당하는 주가로, 실질적으로\n는 기존 대비 17.8% 상향한 셈이며, 목표 P/B는 기존 0.44배에서 0.51배로 상향.동사의 투자 매력으로는 1) 출혈 경쟁 양상으로 진입하는 원수보험 시장에 포함되어\n있지 않다는 점, 2) 경제적 가정이나 계리적 가정으로 인한 피해가 제한적이라는 점,\n3) 앞서 언급한 두 문제로 인해 공동재보험에 대한 원수보험사들의 수요가 증가한다\n는 점을 들 수 있으며, 이런 맥락은 오는 2027년까지 지속될 것으로 예상.더욱이 동사는 안정적인 자본력을 바탕으로 매년 30% 수준의 현금배당과 매 4분기\n무상증자를 규칙적으로 진행해오고 있기 때문에 주주환원의 가시성 측면에서도 뛰어\n나다고 판단. 2024년 및 2025년 예상 배당수익률은 6.0%로 최근 주가 상승에도\n불구, 여전히 경쟁력 있는 수준이라고 판단.2025년 순이익은 전년대비 1.1% 증가할 것으로 예상. 1) 보험손익은 2024년 내내\n반영되었던 해외 생명보험 관련 예실차가 감소하며 전년대비 3.8% 증가, 2) 투자손\n익은 금융상품 평가손익 감소로 전년대비 3.6% 감소할 전망이기 때문. CSM은 공동\n재보험 출재 확대와 조정 감소에 힘입어 전년대비 19.7% 증가할 것으로 예상.\n2025년 K-ICS비율은 195%로, 전년대비 7.2%pt 상승할 전망.3분기 순이익은 670억원으로 당사 추정치 836억원, 컨센서스 734억원 하회. 이는\n환차손 확대에 기인한 것으로, 보험손익은 추정치 부합. K-ICS비율은 전분기대비\n1.6%pt 상승한 187.6%로 추정.주

In [39]:
from langchain_core.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains.combine_documents import (
    create_stuff_documents_chain,
)
from langchain_core.documents import Document

# 요약을 위한 프롬프트 템플릿을 정의합니다.
prompt = PromptTemplate.from_template(
    """아래의 규칙에 따라 차트와 표를 요약해줘
    
규칙:
1. 보기 쉽게 넘버링으로 요약해줘.
2. 한국어로 작성해줘.
3. 기계적 용어는 번역하지 말고, 원문 그대로 보여줘.
4. 불필요한 정보는 포함하지 말아줘.
5. 중요한 실제 값과 수치를 포함해서 요약해줘.

CONTEXT:
{context}

SUMMARY:"
"""
)

# ChatOpenAI 모델의 또 다른 인스턴스를 생성합니다. (이전 인스턴스와 동일한 설정)
llm = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0,
)

# 문서 요약을 위한 체인을 생성합니다.
# 이 체인은 여러 문서를 입력받아 하나의 요약된 텍스트로 결합합니다.
text_summary_chain = create_stuff_documents_chain(llm, prompt)

C:\Users\user\AppData\Local\Temp\ipykernel_34384\4093944364.py:27: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


In [40]:
def create_text_summary(state: GraphState):
    # state에서 텍스트 데이터를 가져옵니다.
    texts = state["texts"]

    # 요약된 텍스트를 저장할 딕셔너리를 초기화합니다.
    text_summary = dict()

    # texts.items()를 페이지 번호(키)를 기준으로 오름차순 정렬합니다.
    sorted_texts = sorted(texts.items(), key=lambda x: x[0])

    # 각 페이지의 텍스트를 Document 객체로 변환하여 입력 리스트를 생성합니다.
    inputs = [
        {"context": [Document(page_content=text)]} for page_num, text in sorted_texts
    ]

    # text_summary_chain을 사용하여 일괄 처리로 요약을 생성합니다.
    summaries = text_summary_chain.batch(inputs)

    # 생성된 요약을 페이지 번호와 함께 딕셔너리에 저장합니다.
    for page_num, summary in enumerate(summaries):
        text_summary[page_num] = summary

    # 요약된 텍스트를 포함한 새로운 GraphState 객체를 반환합니다.
    return GraphState(text_summary=text_summary)


# create_text_summary 함수를 호출하여 텍스트 요약을 생성합니다.
state_out = create_text_summary(state)

# 생성된 요약을 기존 state에 업데이트합니다.
state.update(state_out)

# 요약된 텍스트를 출력합니다.
state_out["text_summary"]

{0: '1. **투자의견**: 매수\n2. **목표주가**: 10,000원 (기존 11,783원 대비 17.8% 상향)\n3. **목표 P/B**: 0.51배 (기존 0.44배에서 상향)\n4. **투자 매력 요인**:\n   - 원수보험 시장에 포함되지 않음\n   - 경제적 및 계리적 가정으로 인한 피해 제한적\n   - 공동재보험에 대한 수요 증가 예상 (2027년까지 지속)\n5. **주주환원**: 매년 30% 수준의 현금배당 및 매 4분기 무상증자\n6. **2024년 및 2025년 예상 배당수익률**: 6.0%\n7. **2025년 순이익 예상**: 전년대비 1.1% 증가\n   - 보험손익: 3.8% 증가\n   - 투자손익: 3.6% 감소\n   - CSM: 19.7% 증가\n8. **2025년 K-ICS비율**: 195% (전년대비 7.2%pt 상승)\n9. **3분기 순이익**: 670억원 (추정치 836억원, 컨센서스 734억원 하회)\n10. **환차손**: 확대된 것으로 추정\n11. **K-ICS비율**: 187.6% (전분기대비 1.6%pt 상승)',
 1: '1. **목표주가 산출**  \n   - 자료 출처: 미래에셋증권 리서치센터  \n   - 목표주가: (구체적인 수치 미제공)\n\n2. **3분기 실적 상세**  \n   - 자료 출처: 미래에셋증권 리서치센터  \n   - 실적: (구체적인 수치 미제공)\n\n3. **분기별 실적 전망**  \n   - 자료 출처: 미래에셋증권 리서치센터  \n   - 전망: (구체적인 수치 미제공)  \n\n*구체적인 수치와 값이 제공되지 않아 요약할 수 있는 정보가 제한적입니다.*',
 2: "1. **회사명**: 코리안리 (003690)\n2. **리서치 기관**: Mirae Asset Securities\n3. **리서치 날짜**: 2024.11.22\n4. **주요 내용**: \n   - 코리안리의 최근 성과 및 전망에 대한 분석.\n   - 시장 동향 및 경쟁사 비교.\n   

## Image, Table 요약을 위한 데이터 배치 생성

In [44]:
state["page_elements"][2]["image_elements"][0]

IndexError: list index out of range

In [45]:
print(state["text_summary"][1])

1. **목표주가 산출**  
   - 자료 출처: 미래에셋증권 리서치센터  
   - 목표주가: (구체적인 수치 미제공)

2. **3분기 실적 상세**  
   - 자료 출처: 미래에셋증권 리서치센터  
   - 실적: (구체적인 수치 미제공)

3. **분기별 실적 전망**  
   - 자료 출처: 미래에셋증권 리서치센터  
   - 전망: (구체적인 수치 미제공)  

*구체적인 수치와 값이 제공되지 않아 요약할 수 있는 정보가 제한적입니다.*


In [46]:
def create_image_summary_data_batches(state: GraphState):
    # 이미지 요약을 위한 데이터 배치를 생성하는 함수
    data_batches = []

    # 페이지 번호를 오름차순으로 정렬
    page_numbers = sorted(list(state["page_elements"].keys()))

    for page_num in page_numbers:
        # 각 페이지의 요약된 텍스트를 가져옴
        text = state["text_summary"][page_num]
        # 해당 페이지의 모든 이미지 요소에 대해 반복
        for image_element in state["page_elements"][page_num]["image_elements"]:
            # 이미지 ID를 정수로 변환
            image_id = int(image_element["id"])

            # 데이터 배치에 이미지 정보, 관련 텍스트, 페이지 번호, ID를 추가
            data_batches.append(
                {
                    "image": state["images"][image_id],  # 이미지 파일 경로
                    "text": text,  # 관련 텍스트 요약
                    "page": page_num,  # 페이지 번호
                    "id": image_id,  # 이미지 ID
                }
            )
    # 생성된 데이터 배치를 GraphState 객체에 담아 반환
    return GraphState(image_summary_data_batches=data_batches)


def create_table_summary_data_batches(state: GraphState):
    # 테이블 요약을 위한 데이터 배치를 생성하는 함수
    data_batches = []

    # 페이지 번호를 오름차순으로 정렬
    page_numbers = sorted(list(state["page_elements"].keys()))

    for page_num in page_numbers:
        # 각 페이지의 요약된 텍스트를 가져옴
        text = state["text_summary"][page_num]
        # 해당 페이지의 모든 테이블 요소에 대해 반복
        for image_element in state["page_elements"][page_num]["table_elements"]:
            # 테이블 ID를 정수로 변환
            image_id = int(image_element["id"])

            # 데이터 배치에 테이블 정보, 관련 텍스트, 페이지 번호, ID를 추가
            data_batches.append(
                {
                    "table": state["tables"][image_id],  # 테이블 데이터
                    "text": text,  # 관련 텍스트 요약
                    "page": page_num,  # 페이지 번호
                    "id": image_id,  # 테이블 ID
                }
            )
    # 생성된 데이터 배치를 GraphState 객체에 담아 반환
    return GraphState(table_summary_data_batches=data_batches)

In [47]:
state_out = create_image_summary_data_batches(state)
state.update(state_out)
state_out

{'image_summary_data_batches': []}

In [62]:
state_out = create_table_summary_data_batches(state)
state.update(state_out)
state_out

KeyError: 'tables'

## Image, Table 요약 추출

In [48]:
from langchain_teddynote.models import MultiModal
from langchain_core.runnables import chain


@chain
def extract_image_summary(data_batches):
    # 객체 생성
    llm = ChatOpenAI(
        temperature=0,  # 창의성 (0.0 ~ 2.0)
        model_name="gpt-4o-mini",  # 모델명
    )

    system_prompt = """
    당신은 이미지에서 유용한 정보를 추출하는 전문가입니다. 주어진 이미지에서 주요 엔터티를 추출하고, 이를 요약하여 나중에 검색에 사용할 수 있는 유용한 정보를 작성하는 것이 당신의 임무입니다.
    """


    image_paths = []
    system_prompts = []
    user_prompts = []

    for data_batch in data_batches:
        context = data_batch["text"]
        image_path = data_batch["image"]
        user_prompt_template = f"""차트와 관련된 내용: {context}
        
###

Output Format:

<image>
<title>
<summary>
<entities> 
</image>

"""
        image_paths.append(image_path)
        system_prompts.append(system_prompt)
        user_prompts.append(user_prompt_template)

    # 멀티모달 객체 생성
    multimodal_llm = MultiModal(llm)

    # 이미지 파일로 부터 질의
    answer = multimodal_llm.batch(
        image_paths, system_prompts, user_prompts, display_image=False
    )
    return answer


@chain
def extract_table_summary(data_batches):
    # 객체 생성
    llm = ChatOpenAI(
        temperature=0,  # 창의성 (0.0 ~ 2.0)
        model_name="gpt-4o-mini",  # 모델명
    )

    system_prompt = """
    당신은 표에서 유용한 정보를 추출하는 전문가입니다. 주어진 이미지에서 주요 엔티티를 추출하고, 이를 요약하여 나중에 검색에 활용할 수 있는 유용한 정보를 작성하는 것이 당신의 임무입니다.
    """

    image_paths = []
    system_prompts = []
    user_prompts = []

    for data_batch in data_batches:
        context = data_batch["text"]
        image_path = data_batch["table"]
        user_prompt_template = f"""표와 관련된 내용: {context}
        
###

Output Format:

<table>
<title>
<table_summary>
<key_entities> 
<data_insights>
</table>

"""
        image_paths.append(image_path)
        system_prompts.append(system_prompt)
        user_prompts.append(user_prompt_template)

    # 멀티모달 객체 생성
    multimodal_llm = MultiModal(llm)

    # 이미지 파일로 부터 질의
    answer = multimodal_llm.batch(
        image_paths, system_prompts, user_prompts, display_image=False
    )
    return answer

In [49]:
def create_image_summary(state: GraphState):
    # 이미지 요약 추출
    # extract_image_summary 함수를 호출하여 이미지 요약 생성
    image_summaries = extract_image_summary.invoke(
        state["image_summary_data_batches"],
    )

    # 이미지 요약 결과를 저장할 딕셔너리 초기화
    image_summary_output = dict()

    # 각 데이터 배치와 이미지 요약을 순회하며 처리
    for data_batch, image_summary in zip(
        state["image_summary_data_batches"], image_summaries
    ):
        # 데이터 배치의 ID를 키로 사용하여 이미지 요약 저장
        image_summary_output[data_batch["id"]] = image_summary

    # 이미지 요약 결과를 포함한 새로운 GraphState 객체 반환
    return GraphState(image_summary=image_summary_output)


# 이미지 요약 생성 함수 실행
state_out = create_image_summary(state)

# 기존 상태 업데이트
state.update(state_out)

# 이미지 요약 결과 출력
state_out["image_summary"]

{}

In [366]:
def create_table_summary(state: GraphState):
    # 테이블 요약 추출
    table_summaries = extract_table_summary.invoke(
        state["table_summary_data_batches"],
    )

    # 테이블 요약 결과를 저장할 딕셔너리 초기화
    table_summary_output = dict()

    # 각 데이터 배치와 테이블 요약을 순회하며 처리
    for data_batch, table_summary in zip(
        state["table_summary_data_batches"], table_summaries
    ):
        # 데이터 배치의 ID를 키로 사용하여 테이블 요약 저장
        table_summary_output[data_batch["id"]] = table_summary

    # 테이블 요약 결과를 포함한 새로운 GraphState 객체 반환
    return GraphState(table_summary=table_summary_output)


# 테이블 요약 생성 함수 실행
state_out = create_table_summary(state)

# 기존 상태 업데이트
state.update(state_out)

# 테이블 요약 결과 출력
state_out["table_summary"]

{10: '<table>\n<title>덕산하이메탈 기업 개요 및 제품 정보</title>\n<table_summary>덕산하이메탈의 설립, 상장, 주요 제품 및 기술 동향에 대한 요약</table_summary>\n<key_entities>\n  <entity>덕산하이메탈</entity>\n  <entity>반도체 패키징</entity>\n  <entity>솔더볼</entity>\n  <entity>솔더페이스트</entity>\n  <entity>Cu Post</entity>\n</key_entities>\n<data_insights>\n  <insight>1999년 설립, 2005년 코스닥 상장</insight>\n  <insight>고순도 금속 정제 기술을 통한 고품질 솔더볼 생산</insight>\n  <insight>소형화 및 고정밀 솔더볼 수요 증가로 시장 성장</insight>\n  <insight>친환경 소재 및 무연 솔더 개발 활발</insight>\n  <insight>450μm에서 40μm 크기 솔더볼 상용화 진행 중</insight>\n</data_insights>\n</table>',
 12: '<table>\n<title>덕산하이메탈 기업 정보 요약</title>\n<table_summary>덕산하이메탈의 기업 개요, 기술 및 제품, 시장 동향, 신제품 개발, 그리고 주요 재무 지표를 요약한 표입니다.</table_summary>\n<key_entities>\n  <entity>덕산하이메탈</entity>\n  <entity>반도체 패키징</entity>\n  <entity>솔더볼</entity>\n  <entity>솔더페이스트</entity>\n</key_entities>\n<data_insights>\n  <insight>현재가: 3,770원</insight>\n  <insight>액면가: 200원</insight>\n  <insight>시가총액: 1,713억 원</insight>\n  <insight>발행주식수: 45,437,00

## Table Markdown 추출

In [53]:
@chain
def table_markdown_extractor(data_batches):
    # 객체 생성
    llm = ChatOpenAI(
        temperature=0,  # 창의성 (0.0 ~ 2.0)
        model_name="gpt-4o-mini",  # 모델명
    )

    system_prompt = "당신은 테이블 이미지의 정보를 마크다운 형식으로 변환하는 전문가입니다. 테이블의 모든 정보를 포함시켜야 하며, 설명은 하지 말고 마크다운 형식으로만 답하십시오"

    image_paths = []
    system_prompts = []
    user_prompts = []

    for data_batch in data_batches:
        image_path = data_batch["table"]
        user_prompt_template = f"""마크다운(```markdown```) 또는 XML 태그로 답변을 감싸지 마세요.
        
###

Output Format:

<table_markdown>

"""
        image_paths.append(image_path)
        system_prompts.append(system_prompt)
        user_prompts.append(user_prompt_template)

    # 멀티모달 객체 생성
    multimodal_llm = MultiModal(llm)

    # 이미지 파일로 부터 질의
    answer = multimodal_llm.batch(
        image_paths, system_prompts, user_prompts, display_image=False
    )
    return answer

In [55]:
def create_table_markdown(state: GraphState):
    # 분석된 JSON 파일 목록을 가져옵니다.
    table_data_batches = []
    
    # 페이지별 테이블 요소 추출
    for page_num in state["page_elements"].keys():
        for table_element in state["page_elements"][page_num]["table_elements"]:
            table_id = table_element["id"]
            if table_id in state["tables"]:
                table_data_batches.append({
                    "table": state["tables"][table_id],
                    "text": state["text_summary"].get(page_num, ""),
                    "page": page_num,
                    "id": table_id,
                })

    # table_markdown_extractor를 사용하여 테이블 마크다운 생성
    table_markdowns = table_markdown_extractor.invoke(table_data_batches)

    # 결과를 저장할 딕셔너리 초기화
    table_markdown_output = dict()

    # 각 데이터 배치와 생성된 테이블 마크다운을 매칭하여 저장
    for data_batch, table_summary in zip(table_data_batches, table_markdowns):
        table_markdown_output[data_batch["id"]] = table_summary

    # 새로운 GraphState 객체 반환
    return GraphState(table_markdown=table_markdown_output)

In [54]:
# def create_table_markdown(state: GraphState):
#     # table_markdown_extractor를 사용하여 테이블 마크다운 생성
#     # state["table_summary_data_batches"]에 저장된 테이블 데이터를 사용
#     table_markdowns = table_markdown_extractor.invoke(
#         state["table_summary_data_batches"],
#     )

#     # 결과를 저장할 딕셔너리 초기화
#     table_markdown_output = dict()

#     # 각 데이터 배치와 생성된 테이블 마크다운을 매칭하여 저장
#     for data_batch, table_summary in zip(
#         state["table_summary_data_batches"], table_markdowns
#     ):
#         # 데이터 배치의 id를 키로 사용하여 테이블 마크다운 저장
#         table_markdown_output[data_batch["id"]] = table_summary

#     # 새로운 GraphState 객체 반환, table_markdown 키에 결과 저장
#     return GraphState(table_markdown=table_markdown_output)


# # create_table_markdown 함수 실행
# state_out = create_table_markdown(state)

# # 기존 state 업데이트
# state.update(state_out)

# # 생성된 테이블 마크다운 출력
# state_out["table_markdown"]

KeyError: 'table_summary_data_batches'

## 결과물

In [56]:
state.keys()

dict_keys(['filepath', 'batch_size', 'split_filepaths', 'analyzed_files', 'page_metadata', 'page_elements', 'page_numbers', 'texts', 'text_summary', 'image_summary_data_batches', 'image_summary'])

In [57]:
state

{'filepath': 'data/pdf/20241122_company_429942000.pdf',
 'batch_size': 1,
 'split_filepaths': ['data/pdf/20241122_company_429942000_0000_0000.pdf',
  'data/pdf/20241122_company_429942000_0001_0001.pdf',
  'data/pdf/20241122_company_429942000_0002_0002.pdf',
  'data/pdf/20241122_company_429942000_0003_0003.pdf'],
 'analyzed_files': ['data/pdf/20241122_company_429942000_0000_0000.json',
  'data/pdf/20241122_company_429942000_0001_0001.json',
  'data/pdf/20241122_company_429942000_0002_0002.json',
  'data/pdf/20241122_company_429942000_0003_0003.json'],
 'page_metadata': {0: {'size': [1241, 1754]},
  1: {'size': [1241, 1754]},
  2: {'size': [1241, 1754]},
  3: {'size': [1241, 1754]}},
 'page_elements': {0: {'image_elements': [],
   'table_elements': [{'bounding_box': [{'x': 52, 'y': 458},
      {'x': 362, 'y': 458},
      {'x': 362, 'y': 1382},
      {'x': 52, 'y': 1382}],
     'category': 'table',
     'html': '<table id=\'2\' style=\'font-size:14px\'><tr><td colspan="2">투자의견(유지)</td><td

In [58]:
state["table_summary"]

KeyError: 'table_summary'

In [59]:
state["table_markdown"]

KeyError: 'table_markdown'

In [60]:
state["image_summary"]

{}

In [61]:
state["texts"]

{0: 'MIRAE ASSET\n미래에셋증권Equity Research\n2024. 11.22[금융]정태준, CFA\ntaejoon.jeong@miraeasset.com003690 · 보험코리안리매력적인 대안투자의견 매수, 목표주가 10,000원 제시코리안리에 대한 투자의견 매수, 목표주가 10,000원 제시. 이는 지난 11월 7일 무\n상증자 신주배정 기준일 이전 기준으로는 11,783원에 해당하는 주가로, 실질적으로\n는 기존 대비 17.8% 상향한 셈이며, 목표 P/B는 기존 0.44배에서 0.51배로 상향.동사의 투자 매력으로는 1) 출혈 경쟁 양상으로 진입하는 원수보험 시장에 포함되어\n있지 않다는 점, 2) 경제적 가정이나 계리적 가정으로 인한 피해가 제한적이라는 점,\n3) 앞서 언급한 두 문제로 인해 공동재보험에 대한 원수보험사들의 수요가 증가한다\n는 점을 들 수 있으며, 이런 맥락은 오는 2027년까지 지속될 것으로 예상.더욱이 동사는 안정적인 자본력을 바탕으로 매년 30% 수준의 현금배당과 매 4분기\n무상증자를 규칙적으로 진행해오고 있기 때문에 주주환원의 가시성 측면에서도 뛰어\n나다고 판단. 2024년 및 2025년 예상 배당수익률은 6.0%로 최근 주가 상승에도\n불구, 여전히 경쟁력 있는 수준이라고 판단.2025년 순이익은 전년대비 1.1% 증가할 것으로 예상. 1) 보험손익은 2024년 내내\n반영되었던 해외 생명보험 관련 예실차가 감소하며 전년대비 3.8% 증가, 2) 투자손\n익은 금융상품 평가손익 감소로 전년대비 3.6% 감소할 전망이기 때문. CSM은 공동\n재보험 출재 확대와 조정 감소에 힘입어 전년대비 19.7% 증가할 것으로 예상.\n2025년 K-ICS비율은 195%로, 전년대비 7.2%pt 상승할 전망.3분기 순이익은 670억원으로 당사 추정치 836억원, 컨센서스 734억원 하회. 이는\n환차손 확대에 기인한 것으로, 보험손익은 추정치 부합. K-ICS비율은 전분기대비\n1.6%pt 상승한 187.6%로 추정.주

In [375]:
state["text_summary"]

{0: '1. **보고서 목적**: 시가총액 5,000억 원 미만의 중소형 기업에 대한 투자정보 제공.\n   \n2. **기업명**: 덕산하이메탈(077360).\n\n3. **작성 기관**: 한국기술신용평가(주).\n\n4. **작성자**: 성재욱 선임연구원.\n\n5. **보고서 내용**:\n   - 기업현황\n   - 시장동향\n   - 기술분석\n   - 재무분석\n   - 주요 변동사항 및 전망\n\n6. **투자 주의사항**: 본 보고서는 참고용으로만 제공되며, 투자자는 자신의 판단과 책임 하에 결정해야 함.\n\n7. **연락처**: 작성기관 문의 전화 - TEL.02-525-7759. \n\n8. **유튜브 및 텔레그램 안내**: 보고서 요약 영상은 유튜브에서 시청 가능하며, 텔레그램 "한국IR협의회" 채널 추가 시 보고서 발간 소식 안내.',
 1: '1. **기업 개요**\n   - 덕산하이메탈(코드: 077360)은 1999년 5월 설립, 2005년 10월 코스닥 상장.\n   - 주 사업: 반도체 패키징용 솔더볼(Solder Ball) 및 솔더페이스트(Solder Paste) R&D, 제조, 판매.\n\n2. **기술 및 제품**\n   - 고순도 금속 정제 기술을 바탕으로 고품질 솔더볼 생산.\n   - 40μm 미만의 초정밀 크기 상용화.\n   - 자체 설계한 제조 설비로 생산공정 최적화 및 불량률 최소화.\n\n3. **시장 동향**\n   - 반도체 패키징 소재 시장은 집적화 및 소형화, 고정밀 솔더볼 수요 증가로 성장 중.\n   - 한국과 일본은 고급 패키징 기술과 국산화에 집중.\n   - 친환경 소재 수요 증가로 무연 솔더 및 지속 가능한 제조 공정 개발 활발.\n\n4. **신제품 개발**\n   - 450μm에서 40μm 크기 솔더볼 상용화, 더 작은 크기 개발 지속.\n   - Cu Post 개발 중, 미세 Pitch 대응 및 다층 구조에서 높은 집적도 구현 가능.\n   - Cu Post는 전기적 성능과 열 관리에

#

In [ ]:
import json

# JSON 형식으로 가독성을 높여 출력
print(json.dumps(state, indent=4, ensure_ascii=False))

{
    "filepath": "C:\\Users\\user\\working\\GitHub\\RePick-MLOps\\data\\pdf\\20241122_company_952985000.pdf",
    "batch_size": 1,
    "split_filepaths": [
        "C:\\Users\\user\\working\\GitHub\\RePick-MLOps\\data\\pdf\\20241122_company_952985000_0000_0000.pdf",
        "C:\\Users\\user\\working\\GitHub\\RePick-MLOps\\data\\pdf\\20241122_company_952985000_0001_0001.pdf",
        "C:\\Users\\user\\working\\GitHub\\RePick-MLOps\\data\\pdf\\20241122_company_952985000_0002_0002.pdf",
        "C:\\Users\\user\\working\\GitHub\\RePick-MLOps\\data\\pdf\\20241122_company_952985000_0003_0003.pdf",
        "C:\\Users\\user\\working\\GitHub\\RePick-MLOps\\data\\pdf\\20241122_company_952985000_0004_0004.pdf",
        "C:\\Users\\user\\working\\GitHub\\RePick-MLOps\\data\\pdf\\20241122_company_952985000_0005_0005.pdf",
        "C:\\Users\\user\\working\\GitHub\\RePick-MLOps\\data\\pdf\\20241122_company_952985000_0006_0006.pdf",
        "C:\\Users\\user\\working\\GitHub\\RePick-MLOps\\data\\pdf

In [ ]:
# import json

# # state를 JSON 파일로 저장
# with open("state.json", "w", encoding="utf-8") as f:
#     json.dump(state, f, ensure_ascii=False, indent=4)

## 멀티-벡터 검색기

In [ ]:
!pip install -U langchain openai chromadb langchain-experimental # 최신 버전이 필요합니다 (멀티 모달을 위해)
!pip install "unstructured[all-docs]" pillow pydantic lxml pillow matplotlib chromadb tiktoken

In [ ]:
# import json
# # state.json 파일을 불러오기
# with open("state.json", "r", encoding="utf-8") as f:
#     state = json.load(f)

In [ ]:
# 파일 경로
fpath = "data/20241121_company_54055000"
fname = "*.png"

### 이미지 요약 - base64로 인코딩

In [247]:
import os
import base64
from glob import glob  # glob 모듈을 추가하여 와일드카드 패턴을 처리합니다.
from langchain_core.messages import HumanMessage

def encode_image(image_path):
    # 이미지 파일을 base64 문자열로 인코딩합니다.
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

def encode_images_in_batches(image_summary_data_batches):
    """
    image_summary_data_batches에 있는 각 이미지 파일을 base64로 인코딩하고,
    인코딩된 결과를 "base64_image" 키로 추가합니다.
    image_summary_data_batches: 이미지와 텍스트 정보가 포함된 배치 리스트
    """
    for batch in image_summary_data_batches:
        image_path = batch["image"]
        # 이미지 파일 경로가 존재하는지 확인
        if os.path.exists(image_path):
            base64_image = encode_image(image_path)
            batch["base64_image"] = base64_image  # base64 인코딩된 이미지를 추가
        else:
            batch["base64_image"] = None  # 이미지 경로가 없으면 None으로 설정
    return image_summary_data_batches

# state에서 "image_summary_data_batches" 가져오기
image_summary_data_batches = state.get("image_summary_data_batches", [])

# 이미지들을 base64로 인코딩
updated_batches = encode_images_in_batches(image_summary_data_batches)

# 인코딩된 결과로 state 업데이트
state["image_summary_data_batches"] = updated_batches

# 결과 확인
for batch in updated_batches:
    print(f"ID: {batch['id']}, Base64 Image: {batch['base64_image'][:50]}...")  # 첫 50자만 출력


ID: 16, Base64 Image: iVBORw0KGgoAAAANSUhEUgAAA+wAAAKgCAIAAACOVmB8AAC2QU...
ID: 19, Base64 Image: iVBORw0KGgoAAAANSUhEUgAAA9wAAAKUCAIAAAB5Tqf/AACrwU...
ID: 23, Base64 Image: iVBORw0KGgoAAAANSUhEUgAABAAAAAK2CAIAAABfPHmGAABVEU...
ID: 28, Base64 Image: iVBORw0KGgoAAAANSUhEUgAAA94AAAKwCAIAAADhhjDiAABXsU...
ID: 30, Base64 Image: iVBORw0KGgoAAAANSUhEUgAACHIAAAOGCAIAAAAFl7L9AAEAAE...
ID: 33, Base64 Image: iVBORw0KGgoAAAANSUhEUgAAA+gAAAKmCAIAAABR5CMbAAEAAE...
ID: 36, Base64 Image: iVBORw0KGgoAAAANSUhEUgAAA7wAAAKYCAIAAADfa2P0AACemk...
ID: 57, Base64 Image: iVBORw0KGgoAAAANSUhEUgAAA3gAAAHqCAIAAADZPXt4AACULk...


In [249]:
state["image_summary_data_batches"]

[{'image': 'data/20241121_company_54055000\\16.png',
  'text': '1. **한국단자의 실적 추이**  \n   - 단위: 십억원, %  \n   - 주요 실적: 매출, 영업이익, 순이익 등\n\n2. **한국단자의 미국 법인 실적**  \n   - 매출액 및 순이익 추이: 연도별 데이터 제공  \n   - 주요 수치: 매출액, 순이익의 변화\n\n3. **한국단자의 멕시코 법인 실적**  \n   - 매출액 및 순이익 추이: 연도별 데이터 제공  \n   - 주요 수치: 매출액, 순이익의 변화\n\n*각 도표의 구체적인 수치는 제공되지 않았습니다.*',
  'page': 1,
  'id': 16,
  'base64_image': '...'},
 {'image': 'data/20241121_company_54055000\\19.png',
  'text': '1. **한국단자의 실적 추이**  \n   - 단위: 십억원, %  \n   - 주요 실적: 매출, 영업이익, 순이익 등\n\n2. **한국단자의 미국 법인 실적**  \n   - 매출액 및 순이익 추이: 연도별 데이터 제공  \n   - 주요 수치: 매출액, 순이익의 변화\n\n3. **한국단자의 멕시코 법인 실적**  \n   - 매출액 및 순이익 추이: 연도별 데이터 제공  \n   - 주요 수치: 매출액, 순이익의 변화\n\n*각 도표의 구체적인 수치는 제공되지 않았습니다.*',
  'page': 1,
  'id': 19,
  'base64_image': '...'},
 {'image': 'data/20241121_company_54055000\\23.png',
  'text': '1. **한국단자의 자동차용 ASP 상승률**: 한국단자의 자동차용 ASP는 지속적으로 상승하고 있으며, 최근 몇 년간의 추세를 보여줌.\n\n2. **국제 구리 가격 추이 (LME Copper)**: LME 구리 가격은 변동성이 있으며, 특정 시점에서 급격한 상승과 하락을 

## 벡터 저장소에 추가하기

In [252]:
state.keys()

dict_keys(['filepath', 'batch_size', 'split_filepaths', 'analyzed_files', 'page_metadata', 'page_elements', 'page_numbers', 'images', 'tables', 'texts', 'text_summary', 'image_summary_data_batches', 'table_summary_data_batches', 'image_summary', 'table_summary', 'table_markdown'])

In [ ]:
import uuid

from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.storage import InMemoryStore
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings


BATCH_SIZE = 300

def create_multi_vector_retriever(
    vectorstore, text_summary, texts, table_summary, tables, table_markdown, image_summary
):
    """
    요약을 색인화하지만 원본 이미지나 텍스트를 반환하는 검색기를 생성합니다.
    """

    # 저장 계층 초기화
    store = InMemoryStore()
    id_key = "doc_id"

    # 멀티 벡터 검색기 생성
    retriever = MultiVectorRetriever(
        vectorstore=vectorstore,
        docstore=store,
        id_key=id_key,
    )

    # 문서를 벡터 저장소와 문서 저장소에 배치 단위로 추가하는 함수
    def add_documents_batch(retriever, doc_summaries, doc_contents):
        # 배치 단위로 처리
        doc_ids = [str(uuid.uuid4()) for _ in doc_contents]  # 문서 내용마다 고유 ID 생성
        
        # 요약 문서 생성
        summary_docs = [
            Document(page_content=str(s), metadata={id_key: doc_ids[i]}) 
            for i, s in enumerate(doc_summaries)
        ]
        
        # 벡터 저장소에 요약 문서 추가
        retriever.vectorstore.add_documents(summary_docs)
        
        # 문서 내용 저장
        retriever.docstore.mset(list(zip(doc_ids, [str(content) for content in doc_contents])))

    # 배치 처리로 문서 추가
    def add_documents_from_state(state, retriever):
        # `text_summary`, `table_summary`, `table_markdown`, `image_summary` 각 항목에 대해 배치 단위로 문서 추가
        for summary, content in [
            (state.get("text_summary", []), state.get("texts", [])),
            (state.get("table_summary_data_batches", []), state.get("tables", [])),
            (state.get("table_markdown", []), state.get("tables", [])),
            (state.get("image_summary_data_batches", []), state.get("images", []))
        ]:
            # 배치 처리 전에 확인: summary와 content가 리스트인지 확인
            if not isinstance(summary, list):
                summary = list(summary.values()) if isinstance(summary, dict) else [str(summary)]
            if not isinstance(content, list):
                content = list(content.values()) if isinstance(content, dict) else [str(content)]
            
            # 배치 처리
            for i in range(0, len(summary), BATCH_SIZE):
                add_documents_batch(retriever, summary[i:i+BATCH_SIZE], content[i:i+BATCH_SIZE])

    # state 객체에서 값을 가져와서 배치 단위로 문서 추가
    add_documents_from_state(state, retriever)

    return retriever


In [ ]:
# 요약을 색인화하기 위해 사용할 벡터 저장소
vectorstore = Chroma(
    collection_name="repick-rag-multi-modal", 
    embedding_function=OpenAIEmbeddings(), 
    persist_directory="./chroma_db"
)

# state 객체에서 필요한 값들 가져오기
text_summary = state.get("text_summary", [])
texts = state.get("texts", [])
table_summary = state.get("table_summary_data_batches", [])
tables = state.get("tables", [])
table_markdown = state.get("table_markdown", [])
image_summary = state.get("image_summary_data_batches", [])


# 검색기 생성
retriever_multi_vector = create_multi_vector_retriever(
    vectorstore,
    text_summary,
    texts,
    table_summary,
    tables,
    table_markdown,
    image_summary
)

# RAG

In [315]:
import io
import re

from IPython.display import HTML, display
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from PIL import Image


def multi_modal_rag_chain(retriever):
    """
    멀티모달 RAG 체인
    """

    # 멀티모달 LLM
    model = ChatOpenAI(temperature=0, model="gpt-4o", max_tokens=2048)

    # RAG 파이프라인
    chain = (
        {
            "context": retriever | RunnableLambda(split_image_text_types),
            "question": RunnablePassthrough(),
        }
        | RunnableLambda(img_prompt_func)
        | model
        | StrOutputParser()
    )

    return chain


# RAG 체인 생성
chain_multimodal_rag = multi_modal_rag_chain(retriever_multi_vector)

## 검사

In [ ]:
# 검색 질의 실행
query = "재무제표를 바탕으로 향후 전망을 알려줘"

# 질의에 대한 문서 6개를 검색합니다.
docs = retriever_multi_vector.invoke(query, limit=6)

# 문서의 개수 확인
len(docs)  # 검색된 문서의 개수를 반환합니다.

4

In [321]:
print(docs[:5])  # docs 리스트의 첫 5개 항목을 출력하여 구조 확인

['한국단자 (025540)도표 1. 한국단자의 실적 추이(단위: 십억원, %)자료: 한국단자, 하나증권도표 2. 한국단자의 미국 법인 매출액/순이익 추이자료: 한국단자, 하나증권도표 3. 한국단자의 멕시코 법인 매출액/순이익 추이자료: 한국단자, 하나증권하나증권·3', '한국단자 (025540)추정 재무제표손익계산서(단위:십억원)대차대조표(단위:십억원)투자지표현금흐름표(단위:십억원)자료: 하나증권하나증권·5', 'data/20241121_company_54055000\\13.png', 'data/20241121_company_54055000\\1.png']


In [324]:
# RAG 체인 실행
print(chain_multimodal_rag.invoke(query))

한국단자(025540)의 최근 재무제표를 분석한 결과, 회사는 3분기 동안 매출액과 영업이익에서 각각 19%와 48%의 연간 증가율을 기록하며 긍정적인 실적을 보였습니다. 특히, 미국 법인의 매출이 317% 증가하며 전체 매출에서 차지하는 비중이 크게 늘어났습니다. 이는 전기차용 ICB(Inter-Connect Board) 제품의 수요 증가와 관련이 있으며, 북미 완성차의 전기차 모델향 납품 증가와 우호적인 환율이 긍정적으로 작용한 결과입니다.

또한, 한국단자의 영업이익률은 12.8%로, 이는 2020년 이후 두 번째로 높은 분기 수익성을 기록한 것입니다. 이는 주요 공장의 가동률 상승과 고정비 레버리지 효과로 인한 매출원가율 감소에 기인합니다. 그러나 영업외 외환손실로 인해 세전이익과 지배주주순이익의 증가율은 영업이익 증가율보다 낮았습니다.

미국과 멕시코 법인의 매출 증가와 수익성 개선은 앞으로도 지속될 것으로 예상되며, 이는 멕시코 2공장의 증설과 신규 고객 발굴에 따른 것입니다. 현재 주가는 P/E 5배대로, 높은 성장성과 수익성 대비 저평가된 상태로 보입니다. 따라서 한국단자는 전기차 시장의 성장과 함께 장기적인 투자 가치가 있는 기업으로 판단됩니다. 투자자들은 이러한 점을 고려하여 투자 결정을 내리는 것이 좋겠습니다.


In [ ]:
# 검색 (테스트)
query = "재무제표를 바탕으로 향후 전망을 알려줘"
results = vectorstore.similarity_search(query, k=2)

for result in results:
    print(result)